Functions for generating the matrices in Supplementary Material C:

In [1]:
def carry(a, b, c): 
    return  (a & b) ^^ (b & c) ^^ (a & c)

def partition(alpha, beta, u, v, a, b, bit_length):
    D = list()
    for i in range(0, 2^bit_length):
        for j in range(0, 2^bit_length):
            i_string = Integer(i).binary().zfill(bit_length)
            j_string = Integer(j).binary().zfill(bit_length)
            x = tuple([Integer(i) for i in i_string])
            y = tuple([Integer(j) for j in j_string])
            
            # Iteratively compute the most significant carries
            k = bit_length
            cx = a
            cy = b
            for t in range(0, k):
                cx = carry(x[k-1-t], y[k-1-t], cx)
                cy = carry(x[k-1-t]^^alpha[-1-t], y[k-1-t]^^beta[-1-t], cy)
                
            if (cx == u) and (cy == v):
                D.append(x + y)
            
    return D

def M(diff_alpha, diff_beta, mask_lambda):
    MM = Matrix(QQ, 4, 4, [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
    for a in [0, 1]:
        for b in [0, 1]:
            for u in [0, 1]:
                for v in [0, 1]:
                    coe = (-1)^(mask_lambda & (diff_alpha ^^ diff_beta ^^ u ^^ v))
                    MM[2*a + b, 2*u + v] = coe * len(partition([diff_alpha], [diff_beta], a, b, u, v, 1))
                    
    return MM

Generate the Matrix $M_{1,0,1}$ in Supplementary Material C:

In [2]:
M(0, 0, 1)

[ 3 -1 -1  1]
[ 0 -2  0  0]
[ 0  0 -2  0]
[ 1 -1 -1  3]

Function for computing the differential-linear correlation of addition modulo $2^n$:

In [3]:
def DL_Correlation(diff_a, diff_b, mask_c, bit_length):
    L = Matrix(QQ, 4, 1, [1, 0, 0, 0,])
    C = Matrix(QQ, 1, 4, [1, 1, 1, 1,])
    
    for k in range(0, bit_length):
        MM = M(diff_a[bit_length-1-k], diff_b[bit_length-1-k], mask_c[bit_length-1-k])
        L = MM * L
    return 2^(-2*bit_length) * C * L

In [4]:
diff_alpha = [1,0,1,1,1,0,1,0,1,1,0,0,0,1,0,0,0,0,1,1,0,1,1,1,1,1,0,0,0,0,0,1]
diff_beta = [1,0,0,0,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,0,1,1,1,0,1,1,0,0,0,0,0,0]
mask_lambda = [1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0]
DL_Correlation(diff_alpha, diff_beta, mask_lambda, 32)

[25/64]

In [5]:
diff_alpha = [1,0,1,1,1,0,1,0,1,1,0,0,0,1,0,0,0,0,1,1,0,1,1,1,1,1,0,0,0,0,0,1]
diff_beta = [1,0,0,0,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,0,1,1,1,0,1,1,0,0,0,0,0,0]

for j in range(0, 31):
    out = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    out[31-j] = 1
    out[31-j-1] = 1
    print(DL_Correlation(diff_alpha, diff_beta, out, 32))

[0]
[1/2]
[3/4]
[7/8]
[15/16]
[31/32]
[1/64]
[-65/128]
[0]
[1/2]
[-3/4]
[0]
[0]
[-1/2]
[0]
[0]
[0]
[0]
[1/2]
[1/4]
[-5/8]
[0]
[0]
[0]
[-1/2]
[0]
[0]
[0]
[0]
[0]
[1/2]


In [6]:
diff_alpha=[1,0,1,1,1,0,1,0,1,1,0,0,0,1,0,0,0,0,1,1,0,1,1,1,1,1,0,0,0,0,0,1,]
diff_beta=[1,0,0,0,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,0,1,1,1,0,1,1,0,0,0,0,0,0,]
for j in range(0,31):
    out = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    out[31-j] = 1
    print(DL_Correlation(diff_alpha, diff_beta, out, 32))

[-1]
[0]
[1/2]
[3/4]
[7/8]
[15/16]
[31/32]
[-1/64]
[65/128]
[0]
[-1/2]
[3/4]
[0]
[0]
[1/2]
[0]
[0]
[0]
[0]
[-1/2]
[1/4]
[-5/8]
[0]
[0]
[0]
[-1/2]
[0]
[0]
[0]
[0]
[0]


64-bit modulo additions

In [7]:
diff_alpha = [1,0,1,1,1,0,1,0,1,1,0,0,0,1,0,0,0,0,1,1,0,1,1,1,1,1,0,0,0,0,0,1,1,0,1,1,1,0,1,0,1,1,0,0,0,1,0,0,0,0,1,1,0,1,1,1,1,1,0,0,0,0,0,1,]
diff_beta = [1,0,1,1,1,0,1,0,1,1,0,0,0,1,0,0,0,0,1,1,0,1,1,1,1,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,0,1,1,1,0,1,1,0,0,0,0,0,0,]
for j in range(0, 63):
    out = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    out[63-j] = 1
    out[63-j-1] = 1
    print(DL_Correlation(diff_alpha, diff_beta, out, 64))

[0]
[1/2]
[3/4]
[7/8]
[15/16]
[31/32]
[1/64]
[-65/128]
[0]
[1/2]
[-3/4]
[0]
[0]
[-1/2]
[0]
[0]
[0]
[0]
[1/2]
[1/4]
[-5/8]
[0]
[0]
[0]
[-1/2]
[0]
[0]
[0]
[0]
[0]
[1/2]
[1/4]
[5/8]
[3/16]
[19/32]
[51/64]
[115/128]
[243/256]
[13/512]
[525/1024]
[1549/2048]
[3597/4096]
[7693/8192]
[499/16384]
[15885/32768]
[48653/65536]
[16883/131072]
[147955/262144]
[410099/524288]
[934387/1048576]
[114189/2097152]
[1982963/4194304]
[6177267/8388608]
[14565875/16777216]
[2211341/33554432]
[35765773/67108864]
[31343091/134217728]
[102874637/268435456]
[165560819/536870912]
[371310093/1073741824]
[1445051917/2147483648]
[3592535565/4294967296]
[702431731/8589934592]


Compute the Rotational Differential-Linear Correlation of Modulo Additions

In [8]:
def RDL_Correlation(diff_a, diff_b, mask_c, bit_length, offset):
    L_1 = Matrix(QQ, 4, 1, [1,0,0,0])
    C_1 = Matrix(QQ, 1, 4, [1,0,1,0])
    L_2 = Matrix(QQ, 4, 1, [0,1,0,0])
    C_2 = Matrix(QQ, 1, 4, [0,1,0,1])
    MC = Matrix(QQ, 4, 4, [1,1,0,0, 0,0,0,0, 0,0,1,1, 0,0,0,0])
    for k in range(offset, bit_length):
        MM = M(diff_a[bit_length-1-k], diff_b[bit_length-1-k], mask_c[bit_length-1-k])
        L_1 = MM * L_1
        L_2 = MM * L_2
    L_1 = MC * L_1
    L_2 = MC * L_2
    for k in range(0, offset):
        MM = M(diff_a[bit_length-1-k], diff_b[bit_length-1-k], mask_c[bit_length-1-k])
        L_1 = MM * L_1
        L_2 = MM * L_2
    return 2^(-2*bit_length) * (C_1 * L_1 + C_2 * L_2)

4.1.1 Set the intput Rx-difference alpha=10111010110001000011011111000001, beta=10000100001001111110111011000000

In [9]:
diff_alpha = [1,0,1,1,1,0,1,0,1,1,0,0,0,1,0,0,0,0,1,1,0,1,1,1,1,1,0,0,0,0,0,1,]
diff_beta = [1,0,0,0,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,0,1,1,1,0,1,1,0,0,0,0,0,0,]
mask_lambda = [1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,]
offset = 30
RDL_Correlation(diff_alpha, diff_beta, mask_lambda, 32, offset)

[838860801/2147483648]

4.2 Show the results of example 6:

In [13]:
diff_alpha = [1,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,0,0,1,1,0,0,1,0]
diff_beta = [1,0,1,0,0,0,0,1,0,1,1,1,0,1,1,1,0,1,0,0,1,1,0,0,0,1,1,1,0,0,1,1]
for j in range(0,31):
    out = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    out[31-j] = 1
    out[31-j-1] = 1
    print(RDL_Correlation(diff_alpha, diff_beta, out, 32, 30))

[0]
[1/2]
[1/4]
[5/8]
[3/16]
[-19/32]
[0]
[1/2]
[3/4]
[-7/8]
[0]
[-1/2]
[0]
[1/2]
[1/4]
[-3/8]
[0]
[-1/2]
[0]
[1/2]
[-1/4]
[0]
[0]
[-1/2]
[0]
[1/2]
[3/4]
[-7/8]
[0]
[1/4294967296]
[1073741825/2147483648]


In [14]:
diff_alpha = [1,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,0,0,1,1,0,0,1,0]
diff_beta = [1,0,1,0,0,0,0,1,0,1,1,1,0,1,1,1,0,1,0,0,1,1,0,0,0,1,1,1,0,0,1,1]
for j in range(0,31):
    out = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    out[31-j] = 1
    print(RDL_Correlation(diff_alpha, diff_beta, out, 32, 30))

[-1/4]
[0]
[-1/2]
[1/4]
[5/8]
[-3/16]
[19/32]
[0]
[1/2]
[3/4]
[-7/8]
[0]
[1/2]
[0]
[1/2]
[-1/4]
[-3/8]
[0]
[1/2]
[0]
[1/2]
[1/4]
[0]
[0]
[-1/2]
[0]
[1/2]
[3/4]
[-7/8]
[0]
[1/1073741824]


64-bit RD-L 

In [15]:
diff_alpha = [1,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,0,0,1,1,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,0,0,1,1,0,0,1,0]
diff_beta = [1,0,1,0,0,0,0,1,0,1,1,1,0,1,1,1,0,1,0,0,1,1,0,0,0,1,1,1,0,0,1,1,1,0,1,0,0,0,0,1,0,1,1,1,0,1,1,1,0,1,0,0,1,1,0,0,0,1,1,1,0,0,1,1]
for j in range(0,63):
    out = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    out[63-j] = 1
    out[63-j-1] = 1
    print(RDL_Correlation(diff_alpha, diff_beta, out, 64, 62))

[0]
[1/2]
[1/4]
[5/8]
[3/16]
[-19/32]
[0]
[1/2]
[3/4]
[-7/8]
[0]
[-1/2]
[0]
[1/2]
[1/4]
[-3/8]
[0]
[-1/2]
[0]
[1/2]
[-1/4]
[0]
[0]
[-1/2]
[0]
[1/2]
[3/4]
[-7/8]
[0]
[1/2]
[1/4]
[-3/8]
[0]
[1/2]
[1/4]
[5/8]
[3/16]
[-19/32]
[0]
[1/2]
[3/4]
[-7/8]
[0]
[-1/2]
[0]
[1/2]
[1/4]
[-3/8]
[0]
[-1/2]
[0]
[1/2]
[-1/4]
[0]
[0]
[-1/2]
[0]
[1/2]
[3/4]
[-7/8]
[0]
[1/18446744073709551616]
[4611686018427387905/9223372036854775808]


4.3 Show the results of example 7:

In [16]:
diff_alpha = [0,1,1,0,0,0,1,1,1,0,1,1,1,0,0,0,1,1,1,1,1,0,1,1,0,1,0,1,0,1,1,1]
diff_beta = [0,1,0,1,0,0,1,1,0,0,1,1,1,1,1,1,1,1,0,1,0,0,1,1,1,1,1,0,0,1,1,1]
for j in range(0,8):
    out = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    out[31-j] = 1
    print(RDL_Correlation(diff_alpha, diff_beta, out, 32, 30))

[1/4]
[-3/8]
[-11/16]
[-27/32]
[-5/64]
[0]
[0]
[1/2]


128-bit DL

In [17]:
diff_alpha = [1,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,0,0,1,1,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,0,0,1,1,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,0,0,1,1,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,0,0,1,1,0,0,1,0]
diff_beta = [1,0,1,0,0,0,0,1,0,1,1,1,0,1,1,1,0,1,0,0,1,1,0,0,0,1,1,1,0,0,1,1,1,0,1,0,0,0,0,1,0,1,1,1,0,1,1,1,0,1,0,0,1,1,0,0,0,1,1,1,0,0,1,1,1,0,1,0,0,0,0,1,0,1,1,1,0,1,1,1,0,1,0,0,1,1,0,0,0,1,1,1,0,0,1,1,1,0,1,0,0,0,0,1,0,1,1,1,0,1,1,1,0,1,0,0,1,1,0,0,0,1,1,1,0,0,1,1]
for j in range(0,127):
    out = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    out[127-j] = 1
    out[127-j-1] = 1
    print(DL_Correlation(diff_alpha, diff_beta, out, 128))

[0]
[1/2]
[1/4]
[5/8]
[3/16]
[-19/32]
[0]
[1/2]
[3/4]
[-7/8]
[0]
[-1/2]
[0]
[1/2]
[1/4]
[-3/8]
[0]
[-1/2]
[0]
[1/2]
[-1/4]
[0]
[0]
[-1/2]
[0]
[1/2]
[3/4]
[-7/8]
[0]
[1/2]
[1/4]
[-3/8]
[0]
[1/2]
[1/4]
[5/8]
[3/16]
[-19/32]
[0]
[1/2]
[3/4]
[-7/8]
[0]
[-1/2]
[0]
[1/2]
[1/4]
[-3/8]
[0]
[-1/2]
[0]
[1/2]
[-1/4]
[0]
[0]
[-1/2]
[0]
[1/2]
[3/4]
[-7/8]
[0]
[1/2]
[1/4]
[-3/8]
[0]
[1/2]
[1/4]
[5/8]
[3/16]
[-19/32]
[0]
[1/2]
[3/4]
[-7/8]
[0]
[-1/2]
[0]
[1/2]
[1/4]
[-3/8]
[0]
[-1/2]
[0]
[1/2]
[-1/4]
[0]
[0]
[-1/2]
[0]
[1/2]
[3/4]
[-7/8]
[0]
[1/2]
[1/4]
[-3/8]
[0]
[1/2]
[1/4]
[5/8]
[3/16]
[-19/32]
[0]
[1/2]
[3/4]
[-7/8]
[0]
[-1/2]
[0]
[1/2]
[1/4]
[-3/8]
[0]
[-1/2]
[0]
[1/2]
[-1/4]
[0]
[0]
[-1/2]
[0]
[1/2]
[3/4]
[-7/8]
[0]
[1/2]
[1/4]


128-bit RDL

In [18]:
diff_alpha = [1,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,0,0,1,1,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,0,0,1,1,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,0,0,1,1,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,0,0,1,1,0,0,1,0]
diff_beta = [1,0,1,0,0,0,0,1,0,1,1,1,0,1,1,1,0,1,0,0,1,1,0,0,0,1,1,1,0,0,1,1,1,0,1,0,0,0,0,1,0,1,1,1,0,1,1,1,0,1,0,0,1,1,0,0,0,1,1,1,0,0,1,1,1,0,1,0,0,0,0,1,0,1,1,1,0,1,1,1,0,1,0,0,1,1,0,0,0,1,1,1,0,0,1,1,1,0,1,0,0,0,0,1,0,1,1,1,0,1,1,1,0,1,0,0,1,1,0,0,0,1,1,1,0,0,1,1]
for j in range(0,127):
    out = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    out[127-j] = 1
    out[127-j-1] = 1
    print(RDL_Correlation(diff_alpha, diff_beta, out, 128, 126))

[0]
[1/2]
[1/4]
[5/8]
[3/16]
[-19/32]
[0]
[1/2]
[3/4]
[-7/8]
[0]
[-1/2]
[0]
[1/2]
[1/4]
[-3/8]
[0]
[-1/2]
[0]
[1/2]
[-1/4]
[0]
[0]
[-1/2]
[0]
[1/2]
[3/4]
[-7/8]
[0]
[1/2]
[1/4]
[-3/8]
[0]
[1/2]
[1/4]
[5/8]
[3/16]
[-19/32]
[0]
[1/2]
[3/4]
[-7/8]
[0]
[-1/2]
[0]
[1/2]
[1/4]
[-3/8]
[0]
[-1/2]
[0]
[1/2]
[-1/4]
[0]
[0]
[-1/2]
[0]
[1/2]
[3/4]
[-7/8]
[0]
[1/2]
[1/4]
[-3/8]
[0]
[1/2]
[1/4]
[5/8]
[3/16]
[-19/32]
[0]
[1/2]
[3/4]
[-7/8]
[0]
[-1/2]
[0]
[1/2]
[1/4]
[-3/8]
[0]
[-1/2]
[0]
[1/2]
[-1/4]
[0]
[0]
[-1/2]
[0]
[1/2]
[3/4]
[-7/8]
[0]
[1/2]
[1/4]
[-3/8]
[0]
[1/2]
[1/4]
[5/8]
[3/16]
[-19/32]
[0]
[1/2]
[3/4]
[-7/8]
[0]
[-1/2]
[0]
[1/2]
[1/4]
[-3/8]
[0]
[-1/2]
[0]
[1/2]
[-1/4]
[0]
[0]
[-1/2]
[0]
[1/2]
[3/4]
[-7/8]
[0]
[1/340282366920938463463374607431768211456]
[85070591730234615865843651857942052865/170141183460469231731687303715884105728]
